In [ ]:
import iris_pipeline

iris_pipeline.monkeypatch_jwst_datamodels()  

In [ ]:
import numpy as np

In [ ]:
from test_utils import get_data_from_url

In [ ]:
from jwst import datamodels

In [ ]:
from iris_pipeline.dark_current.dark_sub import do_correction

## Lower level functions

* `do_correction`

In [ ]:
raw_science_filename = get_data_from_url("17903858")

In [ ]:
input_model = datamodels.open(raw_science_filename)

In [ ]:
input_model

In [ ]:
xstart = 100
ystart = 200
xsize = 50
ysize = 60

In [ ]:
input_model.meta.subarray.name = "CUSTOM"
input_model.meta.subarray.xstart = xstart
input_model.meta.subarray.ystart = ystart
input_model.meta.subarray.xsize = xsize
input_model.meta.subarray.ysize = ysize

In [ ]:
subarray_slice = np.s_[ystart:ystart+ysize, xstart:xstart+xsize]

In [ ]:
input_model.data = input_model.data[subarray_slice]

In [ ]:
input_model.dq = input_model.dq[subarray_slice]

In [ ]:
assert input_model.data.shape == (ysize, xsize)

## High level step class

In [ ]:
step = iris_pipeline.dark_current.DarkCurrentStep()

In [ ]:
step_output = step.run(input_model)

In [ ]:
assert step_output.data.shape == (ysize, xsize)

In [ ]:
step_dark_model = datamodels.open(step.dark_name)

In [ ]:
np.testing.assert_allclose(step_output.data,
                           input_model.data - step_dark_model.data[subarray_slice])